# Config & Imports

In [27]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.callbacks import EarlyStopping

from keras.regularizers import L1L2
from keras.regularizers import L2

import utilities_LR

In [17]:
config = {
    'data': {
        'n_datasets': 10, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 20, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        'n_informative': 10,
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 2,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0.01,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': None,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },    
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae', #tf.keras.losses.get(config['lambda_net']['loss_lambda']),
            'metrics': ['mae']
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 150,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 10,
        'use_gpu': False,
        'gpu_numbers': '4',
        'RANDOM_SEED': 1,   
    }
}

In [18]:
os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [19]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


# Functions

In [20]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config)
    
    with open(directory + '/lambda_weights_list.npy', "rb") as f:
        return np.load(f, allow_pickle=True)

In [21]:
def get_coef():
    # return np.load(utilities_LR.data_path_LR(config) + '/coef_list_targetForInet.npy') ## get coef from sklearn.make_classification
    return np.load(utilities_LR.lambda_path_LR(config) + '/lambda_generated_coef_list_target_for_inet.npy') ## get coef from lambda

In [22]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config['inets']['data_prep']['train_test_val_split']['val_size'] / (config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

In [23]:
def save_model(model):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    model.save(path + '/modelKeras')

# Model Training

In [24]:
X = load_lambda()

In [25]:
y = get_coef()

In [30]:
# Data Prep

early_stopping = EarlyStopping(patience=20, monitor='val_loss')

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                      y)

# Model Def
model = Sequential()
model.add(Dense(6000, activation='relu'))
model.add(Dense(6000, activation='relu'))
model.add(Dense(3500, activation='relu'))
#model.add(Dense(2500, activation='relu'))
#model.add(Dense(1500, activation='linear'))
#model.add(Dense(500, activation='relu'))
model.add(Dense(y.shape[1], activation='linear'))


#opt = tf.keras.optimizers.Adam(
#    learning_rate=3e-4,
#    beta_1=0.999999,
#    beta_2=0.999,
#    epsilon=1e-07,
#    amsgrad=False,
#    name="Adam")

#opt = tf.keras.optimizers.SGD(
#    learning_rate=0.01,
#    momentum=0.0,
#    nesterov=False,
#    name='SGD'
#)

model.compile(optimizer='adam', loss='mse', metrics=['mse', keras.metrics.RootMeanSquaredError()])
model.build(input_shape=X_train.shape)
model.summary()

# Model fit
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=config['inets']['model_fit']['batch_size'],
                    epochs=config['inets']['model_fit']['epochs'],
                    verbose=config['inets']['model_fit']['verbose'],
                    callbacks=[early_stopping],
                    validation_data=(X_val, y_val),
                    shuffle=config['inets']['model_fit']['shuffle'],
                    class_weight=config['inets']['model_fit']['class_weight'],
                    sample_weight=config['inets']['model_fit']['sample_weight'],
                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
                    validation_steps=config['inets']['model_fit']['validation_steps'],
                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
                    validation_freq=config['inets']['model_fit']['validation_freq'],
                   )
print(history.history['val_loss'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (6, 6000)                 49812000  
                                                                 
 dense_11 (Dense)            (6, 6000)                 36006000  
                                                                 
 dense_12 (Dense)            (6, 3500)                 21003500  
                                                                 
 dense_13 (Dense)            (6, 20)                   70020     
                                                                 
Total params: 106,891,520
Trainable params: 106,891,520
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.1947 - mse: 0.1947 - root_mean_squared_error: 0.4412 - val_loss: 534.0380 - val_mse: 534.0380 - val

In [31]:
save_model(model)

INFO:tensorflow:Assets written to: data_LR/nda10_nsa5000_nfe20_nin10_nta1_ncc2_sep1.0_noi0.01_shuTrue_ranNone/tsi0.2_vsi0.1_ranNone_shuTrue_strNone_bat32_epo150_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/tsi0.2_vsi0.1_ranNone_shuTrue_strNone_bat32_epo1000_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/modelKeras/assets
